该notebook用于data exploration，检查数据字段、是否存在缺失值、以及筛选出符合条件的valuation day

In [1]:
import pandas as pd

In [29]:
EXAMPLE_DATA_PATH = "/home/joel/Projects/quant-lab/yield-curve-construction//data/raw/treasuries/crsp_a_treasuries_2018.csv"
data = pd.read_csv(EXAMPLE_DATA_PATH)
data.head()

,KYTREASNO,KYCRSPID,CRSPID,TCUSIP,TDATDT,TMATDT,IWHY,TCOUPRT,TNIPPY,TVALFC,...,TDNOMPRC,TDNOMPRC_FLG,TDSOURCR,TDACCINT,TDRETNUA,TDYLD,TDDURATN,TDPUBOUT,TDTOTOUT,TDPDINT
0,207043,20180104.400000,20180104.400000,912796LC,2017-01-05,2018-01-04,1,0.0,0,0.0,...,99.994083,M,I,0.0,0.000118,0.000030,2.0,NaN,129991.0,0.0
1,207043,20180104.400000,20180104.400000,912796LC,2017-01-05,2018-01-04,1,0.0,0,0.0,...,99.996944,M,I,0.0,0.000029,0.000031,1.0,NaN,129991.0,0.0
2,207105,20180111.400000,20180111.400000,912796ML,2017-07-13,2018-01-11,1,0.0,0,0.0,...,99.971063,M,I,0.0,0.000132,0.000032,9.0,NaN,120000.0,0.0
3,207105,20180111.400000,20180111.400000,912796ML,2017-07-13,2018-01-11,1,0.0,0,0.0,...,99.973556,M,I,0.0,0.000025,0.000033,8.0,NaN,120000.0,0.0
4,207105,20180111.400000,20180111.400000,912796ML,2017-07-13,2018-01-11,1,0.0,0,0.0,...,99.976375,M,I,0.0,0.000028,0.000034,7.0,NaN,120000.0,0.0


In [30]:
data.columns

Index(['KYTREASNO', 'KYCRSPID', 'CRSPID', 'TCUSIP', 'TDATDT', 'TMATDT', 'IWHY',
       'TCOUPRT', 'TNIPPY', 'TVALFC', 'TFCPDT', 'IFCPDTF', 'TFCALDT',
       'TNOTICE', 'IYMCN', 'ITYPE', 'IUNIQ', 'ITAX', 'IFLWR', 'TBANKDT',
       'TSTRIPELIG', 'TFRGNTGT', 'CALDT', 'TDBID', 'TDASK', 'TDNOMPRC',
       'TDNOMPRC_FLG', 'TDSOURCR', 'TDACCINT', 'TDRETNUA', 'TDYLD', 'TDDURATN',
       'TDPUBOUT', 'TDTOTOUT', 'TDPDINT'],
      dtype='object')

In [36]:
print("\n每列的缺失值数量：")
print(data.isnull().sum()[:20])


每列的缺失值数量：
KYTREASNO        0
KYCRSPID         0
CRSPID           0
TCUSIP           0
TDATDT           0
TMATDT           0
IWHY             0
TCOUPRT          0
TNIPPY           0
TVALFC           0
TFCPDT        8646
IFCPDTF          0
TFCALDT      96613
TNOTICE          0
IYMCN        96613
ITYPE            0
IUNIQ            0
ITAX             0
IFLWR            0
TBANKDT      96613
dtype: int64


In [ ]:
print(data.isnull().sum()[20:40])

TSTRIPELIG      96613
TFRGNTGT        96613
CALDT               0
TDBID               0
TDASK               0
TDNOMPRC            0
TDNOMPRC_FLG        0
TDSOURCR            0
TDACCINT            0
TDRETNUA        10559
TDYLD           10438
TDDURATN        10438
TDPUBOUT        10540
TDTOTOUT         1589
TDPDINT             0
dtype: int64


字段介绍

| 角色                 | 字段         |
| ------------------ | ---------- |
| Valuation Date     | `caldt`   |
| 唯一债券 ID         | `CUSIP`    | 
| Issue Date         | `tbankdt`  |
| Maturity Date      | `tmatdt`   |
| Coupon             | `tcouprt`  |
| Clean Price        | `tdnomprc` |
| Accrued Interest   | `tdaccint` |
| Yield (check only) | `tdyld`    |


In [46]:
count_by_date = data.groupby("CALDT")["TCUSIP"].nunique()
count_by_date.describe()

count    249.000000
mean     388.004016
std        4.235505
min      382.000000
25%      385.000000
50%      386.000000
75%      389.000000
max      401.000000
Name: TCUSIP, dtype: float64

In [47]:
candidate_dates = count_by_date.sort_values(ascending=False).head(20)
print(candidate_dates)

CALDT
2018-12-28    401
2018-12-27    400
2018-11-29    399
2018-12-12    399
2018-12-31    399
2018-12-14    398
2018-12-11    398
2018-12-04    398
2018-12-24    398
2018-12-10    398
2018-12-13    398
2018-12-26    398
2018-12-07    398
2018-12-06    398
2018-11-09    397
2018-11-08    397
2018-11-13    397
2018-12-20    397
2018-11-28    397
2018-12-21    397
Name: TCUSIP, dtype: int64


In [52]:
COLS_KEEP = [
    # keys / identifiers
    "CALDT", "KYTREASNO", "TCUSIP", "CRSPID",

    # instrument terms (cashflow)
    "TDATDT", "TFCPDT", "TMATDT", "TCOUPRT", "TNIPPY", "ITYPE",

    # pricing inputs
    "TDNOMPRC", "TDACCINT",

    # liquidity / selection
    "TDBID", "TDASK", "TDPUBOUT", "TDTOTOUT",

    # optional diagnostics
    "TDSOURCR", "TDNOMPRC_FLG", "TDYLD", "TDPDINT", "IWHY"
]



In [58]:
day[COLS_KEEP].isnull().sum()

CALDT            0
KYTREASNO        0
TCUSIP           0
CRSPID           0
TDATDT           0
TFCPDT          43
TMATDT           0
TCOUPRT          0
TNIPPY           0
ITYPE            0
TDNOMPRC         0
TDACCINT         0
TDBID            0
TDASK            0
TDPUBOUT        60
TDTOTOUT        15
TDSOURCR         0
TDNOMPRC_FLG     0
TDYLD           42
TDPDINT          0
IWHY             0
dtype: int64

In [57]:
day.head()

,CALDT,KYTREASNO,TCUSIP,CRSPID,TDATDT,TFCPDT,TMATDT,TCOUPRT,TNIPPY,ITYPE,...,TDACCINT,TDBID,TDASK,TDPUBOUT,TDTOTOUT,TDSOURCR,TDNOMPRC_FLG,TDYLD,TDPDINT,IWHY
11916,2018-12-28,207044,912828U9,20181231.201250,2016-12-31,2017-06-30,2018-12-31,1.250,2,2,...,0.614810,99.976562,100.023438,26003.0,29461.0,I,M,0.000034,0.0,1
12164,2018-12-28,206322,912828RY,20181231.201370,2011-12-31,2012-06-30,2018-12-31,1.375,2,2,...,0.676291,99.984375,100.015625,17240.0,29935.0,I,M,0.000037,0.0,1
12412,2018-12-28,206689,912828A7,20181231.201500,2013-12-31,2014-06-30,2018-12-31,1.500,2,2,...,0.737772,99.976562,100.023438,32935.0,34993.0,I,M,0.000040,0.0,1
12448,2018-12-28,207271,912796UG,20190102.400000,2018-11-08,NaN,2019-01-02,0.000,0,4,...,0.000000,99.970972,99.971111,NaN,30001.0,I,M,0.000058,0.0,1
12697,2018-12-28,207163,912796PK,20190103.400000,2018-01-04,NaN,2019-01-03,0.000,0,4,...,0.000000,99.966583,99.966750,NaN,109982.0,I,M,0.000056,0.0,1


In [56]:
val_date = "2018-12-28"

day = data[data["CALDT"] == val_date].copy()

day = day[COLS_KEEP].copy()


In [59]:
import pandas as pd
import numpy as np

def normalize_types(day: pd.DataFrame) -> pd.DataFrame:
    day = day.copy()

    # 1) dates
    date_cols = ["CALDT", "TDATDT", "TMATDT", "TFCPDT"]
    for c in date_cols:
        if c in day.columns:
            day[c] = pd.to_datetime(day[c], errors="coerce")

    # 2) numeric floats
    float_cols = ["TCOUPRT", "TDNOMPRC", "TDACCINT", "TDBID", "TDASK",
                  "TDPUBOUT", "TDTOTOUT", "TDYLD"]
    for c in float_cols:
        if c in day.columns:
            day[c] = pd.to_numeric(day[c], errors="coerce")

    # 常见哨兵值（如果你的数据里存在的话）
    for c in ["TDYLD"]:
        if c in day.columns:
            day[c] = day[c].replace([-99, -999, 0], np.nan)  # 0 是否当缺失看你版本；不确定就删掉 0

    # 3) ints
    int_cols = ["TNIPPY", "ITYPE", "IWHY"]
    for c in int_cols:
        if c in day.columns:
            day[c] = pd.to_numeric(day[c], errors="coerce").astype("Int64")  # 允许NA的整数

    # 4) ids to string
    str_cols = ["TCUSIP", "CRSPID", "KYTREASNO"]
    for c in str_cols:
        if c in day.columns:
            day[c] = day[c].astype("string")

    return day


day = normalize_types(day)

In [60]:
print(day.dtypes)
print(day[["CALDT","TDATDT","TMATDT"]].head())

CALDT           datetime64[ns]
KYTREASNO       string[python]
TCUSIP          string[python]
CRSPID          string[python]
TDATDT          datetime64[ns]
TFCPDT          datetime64[ns]
TMATDT          datetime64[ns]
TCOUPRT                float64
TNIPPY                   Int64
ITYPE                    Int64
TDNOMPRC               float64
TDACCINT               float64
TDBID                  float64
TDASK                  float64
TDPUBOUT               float64
TDTOTOUT               float64
TDSOURCR                object
TDNOMPRC_FLG            object
TDYLD                  float64
TDPDINT                float64
IWHY                     Int64
dtype: object
           CALDT     TDATDT     TMATDT
11916 2018-12-28 2016-12-31 2018-12-31
12164 2018-12-28 2011-12-31 2018-12-31
12412 2018-12-28 2013-12-31 2018-12-31
12448 2018-12-28 2018-11-08 2019-01-02
12697 2018-12-28 2018-01-04 2019-01-03


In [63]:

day.to_parquet(
    "../data/treasuries_2018-12-28.parquet",
    engine='fastparquet',
    index=False
)